## K-means to segment customer based on RFM

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('Onlineretail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
1,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
2,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
3,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom


In [4]:
type(df)

pandas.core.frame.DataFrame

In [5]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,203422.000000,203422.000000,150039.000000
mean,9.585684,4.877820,15240.234832
std,240.921315,113.686676,1717.658493
min,-74215.000000,0.000000,12346.000000
25%,1.000000,1.250000,13791.000000
50%,3.000000,2.100000,15061.000000
75%,10.000000,4.130000,16771.000000
max,74215.000000,38970.000000,18287.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203422 entries, 0 to 203421
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    203422 non-null  object 
 1   StockCode    203422 non-null  object 
 2   Description  202623 non-null  object 
 3   Quantity     203422 non-null  int64  
 4   InvoiceDate  203422 non-null  object 
 5   UnitPrice    203422 non-null  float64
 6   CustomerID   150039 non-null  float64
 7   Country      203422 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 12.4+ MB


In [8]:
df.isnull().sum()

InvoiceNo          0
StockCode          0
Description      799
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     53383
Country            0
dtype: int64

In [9]:
df.drop(columns=['Description'], inplace=True)
df.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'CustomerID', 'Country'],
      dtype='object')

In [10]:
df[df['Quantity']<0]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
17,539994,22830,-1,2011-01-04 10:09:00,0.00,NaN,United Kingdom
18,539995,85064,-12,2011-01-04 10:12:00,0.00,NaN,United Kingdom
19,539996,21784,-10,2011-01-04 10:12:00,0.00,NaN,United Kingdom
22,539999,20892,-1,2011-01-04 10:16:00,0.00,NaN,United Kingdom
23,540000,84499,-9,2011-01-04 10:17:00,0.00,NaN,United Kingdom
...,...,...,...,...,...,...,...
201945,C558531,22424,-1,2011-06-30 11:31:00,12.75,15039.0,United Kingdom
202318,C558553,23170,-576,2011-06-30 12:44:00,1.45,17949.0,United Kingdom
202427,C558559,20749,-1,2011-06-30 13:20:00,7.95,17648.0,United Kingdom
202627,C558575,22693,-24,2011-06-30 14:58:00,1.25,12674.0,France


In [11]:
df = df[df['Quantity']>0]
df[df['Quantity']<0]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [14]:
df.isnull().sum()

InvoiceNo          0
StockCode          0
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     52480
Country            0
dtype: int64

In [17]:
len(df['InvoiceNo'].unique())

8402

In [18]:
df['InvoiceNo'].unique()

array(['539993', '539997', '539998', ..., '558635', '558636', '558637'],
      dtype=object)

In [23]:
df[df["InvoiceNo"]=='539993']

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,539993,22386,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
1,539993,21499,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
2,539993,21498,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
3,539993,22379,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom
4,539993,20718,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom
5,539993,85099B,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
6,539993,20682,6,2011-01-04 10:00:00,3.25,13313.0,United Kingdom
7,539993,22961,12,2011-01-04 10:00:00,1.45,13313.0,United Kingdom
8,539993,22667,6,2011-01-04 10:00:00,2.95,13313.0,United Kingdom
9,539993,22898,8,2011-01-04 10:00:00,1.95,13313.0,United Kingdom


In [24]:
def fill_invoice(invoice_df):
    valid_customer_ids = invoice_df['CustomerID'].dropna()
    if not valid_customer_ids.empty:
        fill_value = valid_customer_ids.iloc[0]  # Lấy giá trị đầu tiên
        invoice_df['CustomerID'].fillna(fill_value, inplace=True)
    else:
        # Xử lý các InvoiceNo mà tất cả CustomerID đều bị thiếu
        invoice_df['CustomerID'].fillna('Unknown', inplace=True) # Ví dụ điền là Unknown
    return invoice_df
df = df.groupby('InvoiceNo').apply(fill_invoice)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18192\2692739157.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  invoice_df['CustomerID'].fillna(fill_value, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_18192\2692739157.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 198968 entries, ('539993', np.int64(0)) to ('558637', np.int64(203421))
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    198968 non-null  object 
 1   StockCode    198968 non-null  object 
 2   Quantity     198968 non-null  int64  
 3   InvoiceDate  198968 non-null  object 
 4   UnitPrice    198968 non-null  float64
 5   CustomerID   198968 non-null  object 
 6   Country      198968 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 17.6+ MB
